In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_columns", None)
from classes import Paths
paths = Paths()

## 1. `.pivot()`

<div class="alert alert-info">

`.pivot()` returns a transformed DataFrame organized by the specified index/column values without aggregation


<img src="../imgs/03.1.07_1.png" width=600>

In [2]:
path = paths.car_train
df = pd.read_csv(path)
print(df.shape)
df.head(5)

(2337, 10)


,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel


In [3]:
df_pivot = df.pivot(
    index='car_id'
    , columns=['car_type']
    , values=['car_rating']
)
display(df_pivot.head(5))
display(df_pivot.loc[:, [('car_rating', 'economy'), ('car_rating', 'premium')]].head(5))

car_rating                         
car_type     business economy premium standart
car_id                                        
A-1049127W        NaN    4.68     NaN      NaN
A-1079539w        NaN     NaN     NaN     5.40
A-1162143G        NaN    5.24     NaN      NaN
A-1228282M        NaN    4.78     NaN      NaN
A-1339912r        NaN     NaN     NaN     4.88

car_rating        
car_type      economy premium
car_id                       
A-1049127W       4.68     NaN
A-1079539w        NaN     NaN
A-1162143G       5.24     NaN
A-1228282M       4.78     NaN
A-1339912r        NaN     NaN

## 2. `.pivot_table()` with aggregations

In [4]:
df.head(10)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel
5,b12101843B,Skoda Rapid,economy,petrol,2.36,2013,42176,2018,50.93,engine_ignition
6,Q-9368117S,Nissan Qashqai,standart,petrol,5.32,2012,24611,2014,54.79,engine_overheat
7,O-2124190y,Tesla Model 3,premium,electro,3.90,2017,116872,2019,50.26,gear_stick
8,h16895544p,Kia Sportage,standart,petrol,3.50,2014,56384,2017,33.24,gear_stick
9,K77009462l,Smart ForFour,economy,petrol,4.56,2013,41309,2018,39.43,gear_stick


In [5]:
ind_cols = ['car_type']
cols = ['fuel_type']
vals = ['car_rating']

pd.pivot_table(df,
               index=ind_cols,
               columns=cols,
               values=vals,
               aggfunc='mean'
               
)

car_rating       
fuel_type    electro petrol
car_type                   
business        4.44   4.09
economy          NaN   4.36
premium         4.36   4.52
standart         NaN   4.36

<div class="alert alert-info">

If we skip the index, columns, or values, all remaining df columns will be used.  
We can "rotate" the table manually using `groupby`, `stack`, and `unstack`.

<img src="../imgs/03.1.07_2.png" width=600>

<div class="alert alert-info">

Pivot tables are most often used when it is necessary to stretch a categorical feature along the X-axis and calculate various statistics based on it.

<div class="alert alert-success">

By default, the parameter `aggfunc='mean'` is responsible for the aggregation function.  
We can pass a list of functions to it at once. For example, `aggfunc=['mean', 'sum']`.

In [6]:
ind_cols = ['car_type']
cols = ['fuel_type']
vals = ['car_rating']

pd.pivot_table(df,
               index=ind_cols,
               columns=cols,
               values=vals,
               aggfunc=['mean', 'sum']
               
)

mean               sum         
          car_rating        car_rating         
fuel_type    electro petrol    electro   petrol
car_type                                       
business        4.44   4.09      93.26   392.68
economy          NaN   4.36        NaN 6,700.44
premium         4.36   4.52     117.74   262.28
standart         NaN   4.36        NaN 2,609.58

<div class="alert alert-info">

Let's look at the average total and maximum number of drivers who rented cars of a certain class, depending on the year they appeared in the park.:

In [7]:
df_pivot = df.pivot_table(index = 'year_to_start',
                                        columns = ['car_type'],
                                        values = ['riders'],
                                        aggfunc = ['mean', 'sum', 'max']).fillna(0)
df_pivot

mean                                          sum  \
                  riders                                       riders   
car_type        business    economy    premium  standart     business   
year_to_start                                                           
2011            2,577.38   3,552.73       0.00  3,751.49    33,506.00   
2012           23,009.12  23,381.02       0.00 23,193.77   391,155.00   
2013           43,257.07  43,332.51  42,755.69 43,751.05 1,167,941.00   
2014           63,187.09  62,482.09  62,991.20 62,754.52 1,453,303.00   
2015           81,891.45  83,058.95  83,445.88 82,515.11 2,538,635.00   
2016          101,821.00 101,913.71 101,373.41      0.00   101,821.00   
2017          125,625.40 122,085.58 123,482.62      0.00   628,127.00   

                                                              max             \
                                                           riders              
car_type            economy      premium      standart   business    economy   
year_to_start                                                                  
2011             390,800.00         0.00    318,877.00   7,708.00  12,613.00   
2012           3,133,057.00         0.00  2,110,633.00  30,175.00  31,246.00   
2013          11,526,447.00   684,091.00  5,425,130.00  49,634.00  58,090.00   
2014          17,869,878.00   629,912.00  8,283,597.00  76,277.00  77,167.00   
2015          32,392,992.00 1,335,134.00 13,780,023.00  92,250.00  99,295.00   
2016          14,981,316.00 1,723,348.00          0.00 101,821.00 119,546.00   
2017          24,783,373.00 3,210,548.00          0.00 134,530.00 142,862.00   

                                    
                                    
car_type         premium  standart  
year_to_start                       
2011                0.00 18,628.00  
2012                0.00 31,867.00  
2013           54,681.00 53,682.00  
2014           70,494.00 76,591.00  
2015           93,044.00 97,217.00  
2016          111,906.00      0.00  
2017          139,872.00      0.00

<div class="alert alert-info">

The output is a multi-index table that is inconvenient for further use. Let's fix this:

In [8]:
for i in df_pivot.columns:
    print(i)

('mean', 'riders', 'business')
('mean', 'riders', 'economy')
('mean', 'riders', 'premium')
('mean', 'riders', 'standart')
('sum', 'riders', 'business')
('sum', 'riders', 'economy')
('sum', 'riders', 'premium')
('sum', 'riders', 'standart')
('max', 'riders', 'business')
('max', 'riders', 'economy')
('max', 'riders', 'premium')
('max', 'riders', 'standart')


In [9]:
df_pivot.columns = [f'{i[2]}_{i[1]}_{i[0]}' for i in df_pivot.columns]
df_pivot.reset_index(inplace=True)
df_pivot

,year_to_start,business_riders_mean,economy_riders_mean,premium_riders_mean,standart_riders_mean,business_riders_sum,economy_riders_sum,premium_riders_sum,standart_riders_sum,business_riders_max,economy_riders_max,premium_riders_max,standart_riders_max
0,2011,"2,577.38","3,552.73",0.00,"3,751.49","33,506.00","390,800.00",0.00,"318,877.00","7,708.00","12,613.00",0.00,"18,628.00"
1,2012,"23,009.12","23,381.02",0.00,"23,193.77","391,155.00","3,133,057.00",0.00,"2,110,633.00","30,175.00","31,246.00",0.00,"31,867.00"
2,2013,"43,257.07","43,332.51","42,755.69","43,751.05","1,167,941.00","11,526,447.00","684,091.00","5,425,130.00","49,634.00","58,090.00","54,681.00","53,682.00"
3,2014,"63,187.09","62,482.09","62,991.20","62,754.52","1,453,303.00","17,869,878.00","629,912.00","8,283,597.00","76,277.00","77,167.00","70,494.00","76,591.00"
4,2015,"81,891.45","83,058.95","83,445.88","82,515.11","2,538,635.00","32,392,992.00","1,335,134.00","13,780,023.00","92,250.00","99,295.00","93,044.00","97,217.00"
5,2016,"101,821.00","101,913.71","101,373.41",0.00,"101,821.00","14,981,316.00","1,723,348.00",0.00,"101,821.00","119,546.00","111,906.00",0.00
6,2017,"125,625.40","122,085.58","123,482.62",0.00,"628,127.00","24,783,373.00","3,210,548.00",0.00,"134,530.00","142,862.00","139,872.00",0.00


<div class="alert alert-success">

`.pivot_table()` can also be used as an alternative to the `.groupby()` method, grouping values by some attribute with aggregation.:

In [10]:
pivot_table2 = df.pivot_table(index = 'car_id',
                                     #columns = [],
                                     values = ['riders'],
                                     aggfunc = ['mean', 'sum']).fillna(0)
pivot_table2.columns = [f'{i[1]}_{i[0]}' for i in pivot_table2.columns]
pivot_table2.reset_index(inplace=True)
pivot_table2.head()

,car_id,riders_mean,riders_sum
0,A-1049127W,"1,990.00",1990
1,A-1079539w,"76,591.00",76591
2,A-1162143G,"89,458.00",89458
3,A-1228282M,"54,887.00",54887
4,A-1339912r,"65,957.00",65957


<div class="alert alert-warning">

## **Practice Task**: `pivot_table()` and `merge()` *

The `pd.DataFrame()` object is placed in the `df` variable and contains the columns `user_id`, `car_type`, `car_id`, `distance`, `ride_cost`, `speed_max`, `model` (the dataframe is obtained by merging the `car_train` dataset to the `rides_info` dataset).


**Task:** Based on this dataframe, you will need to assemble several pivot tables and then combine them into one final one, getting some kind of embeddings to feed into the model.

- The index in all tables will be `user_id`.
- __The first table__: How many times has a user borrowed cars of a certain class? 
- __The second table__: What were the minimum, maximum, average distance and cost of the user's ride?
- __The third table__: How many unique cars did the user rent?
- Next, we merge all 3 tables into one by key.
- And at the end, we add a binary target to the `crazy_driver` column: Is our user a crazy racer? If its maximum speed exceeded 100 km/h, then 1; if it did not exceed, then 0.
- Write the result of the solution to the result variable.

Comment: Don't forget to get rid of the multiindex and use `pivot_table`. There is no permission to use `.groupby()`. Missing values should be filled with 0.

In [11]:
path = paths.pivot_practice_sample_cars
df = pd.read_csv(path)
df.head(10)

,user_id,car_type,car_id,distance,ride_cost,speed_max,model
0,h14343699V,economy,A-1049127W,158.84,135,72.03,Hyundai Solaris
1,b13580442h,standart,A-1079539w,"3,288.82",510,124.76,Renault Kaptur
2,R21123881y,economy,A-1162143G,"1,464.01",584,47.00,VW Polo
3,P16628111o,standart,A-1079539w,"1,104.89",1148,78.93,Renault Kaptur
4,W15317251O,economy,A-1162143G,983.09,337,180.00,VW Polo
5,u92005384K,economy,A-1162143G,"2,578.13",1179,64.00,VW Polo
6,A66187474K,standart,A-1079539w,"4,621.77",1320,80.40,Renault Kaptur
7,c16243825Y,economy,A-1162143G,"2,139.91",370,75.00,VW Polo
8,e12723895d,standart,A-1079539w,246.50,149,102.00,Renault Kaptur
9,N78592053P,economy,A-1162143G,"1,591.18",746,79.00,VW Polo


**Table 1:**

In [17]:
t1 = pd.pivot_table(df
               , index=['user_id']
               , columns=['car_type']
               , values='car_id'
               , aggfunc=['count'])
for i in t1.columns:
    print(i)
t1.columns = [f'{i[1]}_{i[0]}' for i in t1.columns]
t1.head()

('count', 'economy')
('count', 'standart')


,economy_count,standart_count
user_id,,
A11173530w,1.00,NaN
A66187474K,NaN,1.00
C84438702e,1.00,NaN
E15036556k,NaN,1.00
G84350611k,NaN,1.00


**Table 2:**

In [28]:
pd.pivot_table(
    df
    , index = ['user_id']
    # , columns = ['distance', 'ride_cost']
    , values = ['distance', 'ride_cost']
    , aggfunc = ['min', 'max', 'mean']
)

t2 = pd.pivot_table(
    df
    , index = ['user_id']
    # , columns = ['distance', 'ride_cost']
    , values = ['distance', 'ride_cost']
    , aggfunc = {
        'distance': ['min', 'max', 'mean']
        , 'ride_cost': ['min', 'max', 'mean']
    }
)

for i in t2.columns:
    print(i)

t2.columns = ['{}_{}'.format(i[0], i[1]) for i in t2.columns]
t2.head()

('distance', 'max')
('distance', 'mean')
('distance', 'min')
('ride_cost', 'max')
('ride_cost', 'mean')
('ride_cost', 'min')


,distance_max,distance_mean,distance_min,ride_cost_max,ride_cost_mean,ride_cost_min
user_id,,,,,,
A11173530w,"2,967.60","2,967.60","2,967.60",614,614.00,614
A66187474K,"4,621.77","4,621.77","4,621.77",1320,"1,320.00",1320
C84438702e,"1,221.07","1,221.07","1,221.07",195,195.00,195
E15036556k,"1,284,248.95","1,284,248.95","1,284,248.95",374394,"374,394.00",374394
G84350611k,"1,850.35","1,850.35","1,850.35",428,428.00,428


**Table 3:**

In [32]:
t3 = df.pivot_table(
    index='user_id',
    values='car_id',
    aggfunc=pd.Series.nunique
)
t3.rename(columns={'car_id': 'car_id_nunique'}, inplace=True)
t3.head()

,car_id_nunique
user_id,
A11173530w,1
A66187474K,1
C84438702e,1
E15036556k,1
G84350611k,1


**Merge into one table:**

In [35]:
res = t1.merge(t2, on='user_id').merge(t3, on='user_id')
res.head()

,economy_count,standart_count,distance_max,distance_mean,distance_min,ride_cost_max,ride_cost_mean,ride_cost_min,car_id_nunique
user_id,,,,,,,,,
A11173530w,1.00,NaN,"2,967.60","2,967.60","2,967.60",614,614.00,614,1
A66187474K,NaN,1.00,"4,621.77","4,621.77","4,621.77",1320,"1,320.00",1320,1
C84438702e,1.00,NaN,"1,221.07","1,221.07","1,221.07",195,195.00,195,1
E15036556k,NaN,1.00,"1,284,248.95","1,284,248.95","1,284,248.95",374394,"374,394.00",374394,1
G84350611k,NaN,1.00,"1,850.35","1,850.35","1,850.35",428,428.00,428,1


**Maximum speed more than 100 km/h**

In [37]:
speed_max = pd.pivot_table(
    df
    , index=['user_id']
    # , columns=
    , values=['speed_max']
    , aggfunc='max'
)
res = res.merge(speed_max, on='user_id')
res.head()

,economy_count,standart_count,distance_max,distance_mean,distance_min,ride_cost_max,ride_cost_mean,ride_cost_min,car_id_nunique,speed_max
user_id,,,,,,,,,,
A11173530w,1.00,NaN,"2,967.60","2,967.60","2,967.60",614,614.00,614,1,128.47
A66187474K,NaN,1.00,"4,621.77","4,621.77","4,621.77",1320,"1,320.00",1320,1,80.40
C84438702e,1.00,NaN,"1,221.07","1,221.07","1,221.07",195,195.00,195,1,137.23
E15036556k,NaN,1.00,"1,284,248.95","1,284,248.95","1,284,248.95",374394,"374,394.00",374394,1,62.00
G84350611k,NaN,1.00,"1,850.35","1,850.35","1,850.35",428,428.00,428,1,76.00


In [38]:
lf = lambda x: 1 if x.speed_max > 100 else 0
res['crazy_driver'] = res.apply(lf
                                , axis=1)
res.head()

# alternative way to add a column
# pivot_table['crazy_driver'] = (pivot_table['speed'] > 100).astype(int)

,economy_count,standart_count,distance_max,distance_mean,distance_min,ride_cost_max,ride_cost_mean,ride_cost_min,car_id_nunique,speed_max,crazy_driver
user_id,,,,,,,,,,,
A11173530w,1.00,NaN,"2,967.60","2,967.60","2,967.60",614,614.00,614,1,128.47,1
A66187474K,NaN,1.00,"4,621.77","4,621.77","4,621.77",1320,"1,320.00",1320,1,80.40,0
C84438702e,1.00,NaN,"1,221.07","1,221.07","1,221.07",195,195.00,195,1,137.23,1
E15036556k,NaN,1.00,"1,284,248.95","1,284,248.95","1,284,248.95",374394,"374,394.00",374394,1,62.00,0
G84350611k,NaN,1.00,"1,850.35","1,850.35","1,850.35",428,428.00,428,1,76.00,0


In [40]:
result = res.copy()
result.head(10).reset_index()

,user_id,economy_count,standart_count,distance_max,distance_mean,distance_min,ride_cost_max,ride_cost_mean,ride_cost_min,car_id_nunique,speed_max,crazy_driver
0,A11173530w,1.00,NaN,"2,967.60","2,967.60","2,967.60",614,614.00,614,1,128.47,1
1,A66187474K,NaN,1.00,"4,621.77","4,621.77","4,621.77",1320,"1,320.00",1320,1,80.40,0
2,C84438702e,1.00,NaN,"1,221.07","1,221.07","1,221.07",195,195.00,195,1,137.23,1
3,E15036556k,NaN,1.00,"1,284,248.95","1,284,248.95","1,284,248.95",374394,"374,394.00",374394,1,62.00,0
4,G84350611k,NaN,1.00,"1,850.35","1,850.35","1,850.35",428,428.00,428,1,76.00,0
5,I12505308H,NaN,1.00,"1,023.81","1,023.81","1,023.81",509,509.00,509,1,56.55,0
6,N78592053P,1.00,NaN,"1,591.18","1,591.18","1,591.18",746,746.00,746,1,79.00,0
7,O16691347p,1.00,NaN,229.52,229.52,229.52,202,202.00,202,1,90.00,0
8,P16628111o,NaN,1.00,"1,104.89","1,104.89","1,104.89",1148,"1,148.00",1148,1,78.93,0
9,R16443619c,1.00,NaN,"1,324.07","1,324.07","1,324.07",512,512.00,512,1,83.00,0


In [ ]:
# YOUR CODE HERE
t1 = pd.pivot_table(df
               , index=['user_id']
               , columns=['car_type']
               , values='car_id'
               , aggfunc=['count'])

t1.columns = [f'{i[1]}_{i[0]}' for i in t1.columns]


t2 = pd.pivot_table(
    df
    , index = ['user_id']
    # , columns = ['distance', 'ride_cost']
    , values = ['distance', 'ride_cost']
    , aggfunc = {
        'distance': ['min', 'max', 'mean']
        , 'ride_cost': ['min', 'max', 'mean']
    }
)

t2.columns = ['{}_{}'.format(i[0], i[1]) for i in t2.columns]

t3 = df.pivot_table(
    index='user_id',
    values='car_id',
    aggfunc=pd.Series.nunique
)
t3.rename(columns={'car_id': 'car_id_nunique'}, inplace=True)

res = t1.merge(t2, on='user_id').merge(t3, on='user_id')

speed_max = pd.pivot_table(
    df
    , index=['user_id']
    # , columns=
    , values=['speed_max']
    , aggfunc='max'
)
res = res.merge(speed_max, on='user_id')

lf = lambda x: 1 if x.speed_max > 100 else 0
res['crazy_driver'] = res.apply(lf
                                , axis=1)

result = res.copy().reset_index().fillna(0).drop(columns=['speed_max'], axis=1)